In [1]:
import os
import email
import numpy as np
import pandas as pd
from nltk.tokenize.regexp import RegexpTokenizer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd

In [4]:
df = pd.read_csv('/content/Electoral_Analysis.csv')
df

,Unnamed: 0,Tweet,Label
0,0,@anjanaomkashyap I am seeing you as future #bj...,0
1,1,#LokSabhaElections2019 \n23rd May 2019 will re...,0
2,2,#LokSabhaElections2019 \n23rd May 2019 will re...,0
3,3,PM Modi creates a new record of being the only...,0
4,4,@abhijitmajumder Appointment of Successor! \n\...,0
...,...,...,...
39944,39944,"@quizzicalguy In this #LokSabhaElections2019, ...",1
39945,39945,@AnumaVidisha @RahulGandhi @ArvindKejriwal whe...,1
39946,39946,@sherryontopp Early morning #political move af...,1
39947,39947,@mayankgandhi04 @mallesh_2004 Early morning #p...,1


In [13]:
import torch
from sklearn.model_selection import train_test_split as tts
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd

# Load data
df = pd.read_csv('/content/Electoral_Analysis.csv')

train_data, test_data = tts(df, test_size = 0.2, random_state = 42)
#train_data = train_data[['text', 'label']]
#test_data = test_data[['text', 'label']]

#retain_data, forget_data = train_test_split(train_data, test_size=0.4, random_state=42)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')

# Load model
PATH = '/content/drive/MyDrive/Unlearning-Model-Weights/Elect_Bert.pt' #weights or checkpoints
model = torch.load(PATH)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu")) #gpu pe assign krna model ko

# Define batch size
batch_size = 8

# Move data to the same device as the model
def process_data(data, model_device):
    data_text = data['Tweet'].tolist()
    data_label = data['Label'].tolist()

    encodings = tokenizer(data_text, truncation=True, padding=True, max_length=256, return_tensors='pt').to(model_device)
    labels = torch.tensor(data_label).to(model_device)

    return encodings, labels

# Process data in batches
#retain_batches = [process_data(retain_data[i:i+batch_size], model.device) for i in range(0, len(retain_data), batch_size)]
#forget_batches = [process_data(forget_data[i:i+batch_size], model.device) for i in range(0, len(forget_data), batch_size)]
test_batches = [process_data(test_data[i:i+batch_size], model.device) for i in range(0, len(test_data), batch_size)]

# Evaluate model in batches
model.eval()
with torch.no_grad():
    #retain_embeddings = torch.cat([model(**encodings).logits[:, 0].reshape(-1, 1) for encodings, _ in retain_batches], dim=0)
    #forget_embeddings = torch.cat([model(**encodings).logits[:, 0].reshape(-1, 1) for encodings, _ in forget_batches], dim=0)
    test_embeddings = torch.cat([model(**encodings).logits[:, 0].reshape(-1, 1) for encodings, _ in test_batches], dim=0)

# Train Logistic Regression
classifier = LogisticRegression()
classifier.fit(test_embeddings.cpu().numpy(), test_data['Label'])

# Make predictions on test data
test_predictions = classifier.predict(test_embeddings.cpu().numpy()) # 0 or 1.

#for i in range(len(df)):
  #print(df['Tweet'][i])
  #print("Model Predicted results", test_predictions[i], "Actual Restuls", test_data['Label'][i])

# Calculate accuracy

accuracy = accuracy_score(test_data['Label'], test_predictions)

print("--------Accuracy------")
print(accuracy)

--------Accuracy------
0.9076345431789737
